# Nettoyage et exploration des données en lien avec les bus et tram d'Angers 

## 1. Import des packages principaux

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings

import requests 
import json

warnings.filterwarnings('ignore')
%matplotlib inline

import dtale

### 2.1 Import des données à partir de l'api de la ville d'Angers

In [ ]:
r = requests.get("https://data.angers.fr/api/records/1.0/search/",
    params = {    
        "dataset":"bus-tram-position-tr",
        "rows":-1,    
    },
)

r.raise_for_status()
    
d = r.json()
#d

In [ ]:
df_api = pd.json_normalize(d['records'])

### 2.2 Import des données à partir de la base consolidées des différents .json

In [ ]:
df = pd.read_csv("angers_bus_tram_json_merge.csv", sep=";", encoding="utf-8")
df.sample(5)

## 3. Nettoyage des données 

On supprime les colonnes inutiles et on transforme les types de données.

In [ ]:
df = (df
        .drop(columns=["Unnamed: 0", "datasetid", "recordid", "geometry.type"], errors='ignore')
        .astype({"record_timestamp": "datetime64", "fields.ts_maj": "datetime64", "fields.coordonnees" : "string"})
        .assign(ecart_horodatage = lambda x: x["record_timestamp"] - x["fields.ts_maj"])
)


On renomme les colonnes pour une meilleure lecture et utilisation des métadonnés.

In [ ]:
nomsColonnes = ['horodatage', 
                'identifiant_SAE_de_desserte',
                'identifiant_du_vehicule',
                'numero_Timeo_de_l_arret',
                'etat_SAE_du_vehicule', 
                'numero_de_parc_du_vehicule',
                'nom_de_la_ligne',
                'Heure_estimee_de_passage_a_L_arret', 
                'modele_du_vehicule',
                'identifiant_SAE_de_ligne',
                'ecart_horaire_en_secondes',
                'destination',
                'nom_de l_arret', 
                'mne_de_l_arret',
                'mnemo_de_la_ligne',
                 'coordonnees_GPS_WG84', 
                'identifiant_SAE_de_l_arret',
                'cap_du_vehicule_en_degres', 
                'identifiant_SAE_du_parcours',
                'service_voiture', 
                'coordonnees_GPS_X',
                'coordonnees_GPS_Y', 
                'horodatage_maj',
                'cordonnees_bus_geometrie', 
                'ecart_horodatage']


df.columns = nomsColonnes

On supprime les doublons qui vont fausser nos analyses. 

In [ ]:
# index des lignes dupliquées (on drop les unhashable)
duplicateRowsDF = df[df.drop(columns=["cordonnees_bus_geometrie", "coordonnees_GPS_WG84"]).duplicated()]
#display(duplicateRowsDF.sort_values(by="horodatage").head(10))

# drop des lignes dupliquées par index
df = df.drop(duplicateRowsDF.index, axis=0)
df.shape


On regarde le nombres de NA, ici 14 770.

Actuellement, la décision est de les supprimer.

In [ ]:
print("Nbr NA avant suppression : ", df.isna().sum().sum())

df = df.dropna()

print("Nbr NA avant suppression :", df.isna().sum().sum())

On va s'occupe des données abérantes (outsiders). 

On utilise la méthode de winsorizing sur les données : 
- inférieur à q1 - 1.5 * irq,
- supérieur à q3 + 1.5 * irq. 



In [ ]:
from scipy.stats.mstats import winsorize

fig = px.box(df, y="ecart_horaire_en_secondes")
fig.show()

df["ecart_horaire_en_secondes"] = winsorize(df["ecart_horaire_en_secondes"], limits=[0.05, 0.05])

fig = px.box(df, y="ecart_horaire_en_secondes")
fig.show()

On développe la nootion des coordonnées GPS, l'idée étant de séparer la latitude et la longitude. 

In [ ]:
df[['latitude', 'longitude']] = df['coordonnees_GPS_WG84'].str.split(',', expand=True)

cols = ['latitude', 'longitude']
for col in cols :
    df[col] = df[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)


De plus, on divise la colonne horodatage en plusieurs colonnes numériques qui vont être utile à l'affichage des analyses :
On récupère :
- l'année,
- le mois,
- le jour,
- la date sur un nouveau format, 
- la date avec l'heure,
- le jour de la semaine. 

In [ ]:
df["year"] = df["horodatage"].dt.year
df["month"] = df["horodatage"].dt.month
df["day"] =  df["horodatage"].dt.day
df["hours"] = df["horodatage"].dt.hour
df["date"] = pd.to_datetime(df[["year", "month", "day"]])
df["date_heure"] = pd.to_datetime(df[["year", "month", "day", "hours"]])

df["jour_semaine"] = df["date"].dt.day_name().map({"Monday": "Lundi", "Tuesday": "Mardi", "Wednesday": "Mercredi", "Thursday": "Jeudi", "Friday": "Vendredi", "Saturday": "Samedi", "Sunday": "Dimanche"})

#df.sample(5)

On en arrive à un dataframe de la forme suivante, avec 25 colonnes et 724 202 observations. 

In [ ]:
print(df.shape)
#dtale.show(df)

On affiche les informations de nos données. 

In [ ]:
df.info()

## 4 Analyse de nos données 

### 4.1 Analyse générale 

In [ ]:
print("Premiere date : ", df["horodatage"].min().strftime("%Y-%m-%d"))
print("Derniere date : ", df["horodatage"].max().strftime("%Y-%m-%d"))
print("Nombre de jours étudiés : ", (df["horodatage"].max() - df["horodatage"].min()).days)
print("Nombre de véhicules différents : ", df["identifiant_du_vehicule"].nunique())
print("Nombre de lignes différentes : ", df["nom_de_la_ligne"].nunique())


### 4.2 Analyse des retard => écart horaire 

In [ ]:
# nombre de bus avec ecart_hoiraire_en_seconde > 0 
tab_ecart = [len(df[df["ecart_horaire_en_secondes"] > 0]), len(df[df["ecart_horaire_en_secondes"] < 0]) , len(df[df["ecart_horaire_en_secondes"] == 0])]
tab_label = ["Vehicule en retard", "Véhicule en avance", "Véhicule à l'heure"]

fig = go.Figure(data=[go.Pie(labels=tab_label, values=tab_ecart, pull = [0,0.2,0])])
fig.show()


In [ ]:
ecart_jour = df.groupby(['month'])['ecart_horaire_en_secondes'].count()

#ecart_jour.plot(kind='bar',figsize=(12,7), color='magenta', alpha=0.5)$
fig = px.bar(ecart_jour, 
            x=ecart_jour.index, 
            y='ecart_horaire_en_secondes', 
            title="Nombre de retard par mois en seconde",
            color='ecart_horaire_en_secondes')
fig.show()

In [ ]:
df_plot_ecart = df[['month', 'day', 'ecart_horaire_en_secondes']]
df_plot_ecart = df_plot_ecart.groupby(['month', 'day']).sum().reset_index()
#display(df_plot_ecart)

df_plot_ecart_8 = df_plot_ecart[df_plot_ecart['month'] == 8]
df_plot_ecart_9 = df_plot_ecart[df_plot_ecart['month'] == 9]
df_plot_ecart_10 = df_plot_ecart[df_plot_ecart['month'] == 10]
df_plot_ecart_11 = df_plot_ecart[df_plot_ecart['month'] == 11]
df_plot_ecart_12 = df_plot_ecart[df_plot_ecart['month'] == 12]

fig = make_subplots(rows=2, cols=3)
fig.add_trace(go.Line(x=df_plot_ecart_8['day'], y=df_plot_ecart_8['ecart_horaire_en_secondes'], name = "Août"), row=1, col=1)
fig.add_trace(go.Line(x=df_plot_ecart_9['day'], y=df_plot_ecart_9['ecart_horaire_en_secondes'], name = "Septembre"), row=1, col=2)
fig.add_trace(go.Line(x=df_plot_ecart_10['day'], y=df_plot_ecart_10['ecart_horaire_en_secondes'], name = "Octobre"), row=1, col=3)
fig.add_trace(go.Line(x=df_plot_ecart_11['day'], y=df_plot_ecart_11['ecart_horaire_en_secondes'], name = "Novembre"), row=2, col=1)
fig.add_trace(go.Line(x=df_plot_ecart_12['day'], y=df_plot_ecart_12['ecart_horaire_en_secondes'], name = "Décembre"), row=2, col=2)
fig.update_layout(yaxis_range=[0, 2500000])
fig.show()

In [ ]:
fig = px.bar(df_plot_ecart, 
            x='day',
            y='ecart_horaire_en_secondes',
            animation_frame="month",
            range_y=[0,2500000],
            )
fig.show()

In [ ]:
df_heatmap_ecart = df[['month', 'day', 'ecart_horaire_en_secondes']]
df_heatmap_ecart = df_heatmap_ecart.groupby(['month', 'day']).sum().reset_index()
df_heatmap_ecart = df_heatmap_ecart.pivot("month", "day", "ecart_horaire_en_secondes")
df_heatmap_ecart = df_heatmap_ecart.fillna(0)
df_heatmap_ecart.index = ["Août", "Septembre", "Octobre", "Novembre", "Décembre"]

px.imshow(df_heatmap_ecart, labels=dict(x="Jour", y="Mois", color="Ecart en seconde", height=1000, width=500, aspect="auto"))

### Analyse des différents modèles de véhicules

In [ ]:
# repartition identifiant et modele du vehicule
df_plot_vehicule_repartition = df[['identifiant_du_vehicule', 'modele_du_vehicule']].drop_duplicates()
df_plot_vehicule_repartition["count"] = 1
df_plot_vehicule_repartition = df_plot_vehicule_repartition.groupby(['modele_du_vehicule']).sum().reset_index().drop(columns=["identifiant_du_vehicule"])

fig = px.bar(df_plot_vehicule_repartition, 
            x='modele_du_vehicule', 
            y='count', 
            title="Répartition des différents modèles de véhicules",
            color='count')
fig.show()

### Analyse de l'état des véhicules 

> 'HLP', course en Haut le pied (course sans client, entre dépôt et terminus)
> 'TARR', Terminus arrivée
> 'TDEP' Terminus Départ
> 'LIGN' En ligne commerciale

In [ ]:
df_plot_vehicule_etat = df[['identifiant_du_vehicule', 'etat_SAE_du_vehicule', 'month', 'day']].drop_duplicates()
df_plot_vehicule_etat["count"] = 1
df_plot_vehicule_etat = df_plot_vehicule_etat.groupby(['month', 'day', 'etat_SAE_du_vehicule']).sum().reset_index().drop(columns=["identifiant_du_vehicule"])
#display(df_plot_vehicule_etat)

fig = px.bar(df_plot_vehicule_etat, x='day', y='count', animation_frame="month", color='etat_SAE_du_vehicule')
fig.show()

### Analyse des différentes lignes de bus / tram 

In [ ]:
# nombre de bus moyen par lignes
df_plot_bus_par_ligne = df[['date', 'month', 'identifiant_du_vehicule', 'mnemo_de_la_ligne']].drop_duplicates()
df_plot_bus_par_ligne["count"] = 1
df_plot_bus_par_ligne = df_plot_bus_par_ligne.groupby(['date', 'month', 'mnemo_de_la_ligne']).sum().reset_index().drop(columns=["identifiant_du_vehicule"])
display(df_plot_bus_par_ligne)

# moyenne par mnemo_de_la_ligne
df_plot_bus_par_ligne_moyenne = df_plot_bus_par_ligne.groupby(['mnemo_de_la_ligne']).mean().round(0).reset_index().drop(columns=["month"])
#display(df_plot_bus_par_ligne_moyenne)

# moyenne de mnemo_de_la_ligne par mois
df_plot_bus_par_ligne_moyenne_par_mois = df_plot_bus_par_ligne.groupby(['month', 'mnemo_de_la_ligne']).mean().round(0).reset_index()
#display(df_plot_bus_par_ligne_moyenne_par_mois.sort_values(by=["month","mnemo_de_la_ligne"]))


In [ ]:
list_unique_mnemo_de_la_ligne = df_plot_bus_par_ligne.mnemo_de_la_ligne.unique().tolist()

button_list = []
for items in list_unique_mnemo_de_la_ligne:
    button_list.append(dict(label=items,
                            method="update",
                            args=[{"visible": [items in l for l in df_plot_bus_par_ligne["mnemo_de_la_ligne"]]},
                                  {"title": "Nombre de bus moyen par lignes : {}".format(items)}]))

# plotly drowpdown from list_unique_mnemo_de_la_ligne
fig = px.line(df_plot_bus_par_ligne, x="date", y="count", color='mnemo_de_la_ligne', title="Nombre de bus moyen par lignes")
fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons = button_list,
            direction = "down",
            pad={"r": 10, "t": 10},
            showactive = True
        )
    ]
)
fig.show()


In [ ]:
# ecart horaire par ligne
df_plot_ecart_horaire_par_ligne = df[['date_heure', 'month', 'identifiant_du_vehicule', 'mnemo_de_la_ligne', 'ecart_horaire_en_secondes']]
display(df_plot_ecart_horaire_par_ligne)


In [ ]:
df_plot_ecart_horaire_par_ligne_all = df_plot_ecart_horaire_par_ligne
df_plot_ecart_horaire_par_ligne_all["count"] = 1
df_plot_ecart_horaire_par_ligne_all = df_plot_ecart_horaire_par_ligne_all.groupby(['mnemo_de_la_ligne']).sum().reset_index().drop(columns=['month','identifiant_du_vehicule'])
df_plot_ecart_horaire_par_ligne_all["moyenne_retard"] = round(df_plot_ecart_horaire_par_ligne_all["ecart_horaire_en_secondes"] / df_plot_ecart_horaire_par_ligne_all["count"],0)

display(df_plot_ecart_horaire_par_ligne_all)

# passage en minute
# affichage globale

# affichage heatmap en fct des mois / ligne




### Implémentation géographique des différents bus / trams en fonction du temps 

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMapWithTime
from folium.plugins import HeatMap
import datetime

def generateBaseMap(default_location=[47.478419, -0.563166], default_zoom_start=11):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=14)
    return base_map

base_map = generateBaseMap()


In [ ]:
df_map = df[['date_heure', 'latitude', 'longitude']]
df_map["count"] = 1
df_map['count'] = df_map[['date_heure', 'latitude', 'longitude', 'count']].groupby(['latitude', 'longitude', 'date_heure']).transform('count')

#df_map.head(10)

In [ ]:
df_map['date_heure']= pd.to_datetime(df_map['date_heure'])
temps_index = []
for i in df_map['date_heure'].unique():
    temps_index.append(i)
date_labels = [pd.to_datetime(str(d)).strftime('%d/%m/%Y, %H') for d in temps_index]

date_labels = [x for _,x in sorted(zip(temps_index,date_labels))]
date_labels


lat_long_list = []
for i in df_map['date_heure'].unique():
    temp=[]
    for index, instance in df_map[df_map['date_heure'] == i].iterrows():
        temp.append([instance['latitude'],instance['longitude']])
    lat_long_list.append(temp)

In [ ]:
base_map = generateBaseMap()

#cluster = plugins.MarkerCluster().add_to(base_map)

#HeatMap(data=df_map[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright',name="cluster", index=date_labels, max_opacity=0.9).add_to(base_map)


base_map